In [ ]:
'''
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session'''

'\n# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python\n# For example, here\'s several helpful packages to load\n\nimport numpy as np # linear algebra\nimport pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\n\n# Input data files are available in the read-only "../input/" directory\n# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory\n\nimport os\nfor dirname, _, filenames in os.walk(\'/kaggle/input\'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n\n# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" \n# You can also write temporary files to /kaggle/temp/, but they won\'t be saved outside of the current session'

In [ ]:
import numpy as np
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import OneHotEncoder

data_path = '/kaggle/input/flowers-recognition/flowers'

In [ ]:
def load_images(data_path, image_size=(32, 32)):
    images_data = []
    labels_data = []
    label_dict = {'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4}
    for folder_name in os.listdir(data_path):
        if folder_name in label_dict:
            folder_path = os.path.join(data_path, folder_name)
            for file_name in os.listdir(folder_path):
                image_path = os.path.join(folder_path, file_name)
                with Image.open(image_path) as image_file:
                    image_file = image_file.resize(image_size).convert('RGB')
                    images_data.append(np.asarray(image_file).flatten() / 255.0)
                    labels_data.append(label_dict[folder_name])
    return np.array(images_data), np.array(labels_data).reshape(-1, 1)

In [ ]:
images_data, labels_data = load_images(data_path)
X_train, X_test, y_train, y_test = train_test_split(images_data, labels_data, test_size=0.2, random_state=42)

In [ ]:
encoder = OneHotEncoder(sparse=False)
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
input_size = 32 * 32 * 3
hidden_units = [128, 64]
output_size = 5
learning_rate = 0.01
epochs = 10

In [ ]:
def initialize_weights(input_size, hidden_units, output_size):
    np.random.seed(42)
    layer_units = hidden_units + [output_size]
    weights = {}
    biases = {}

    for i in range(len(layer_units)):
        weights[i] = np.random.randn(input_size if i == 0 else layer_units[i-1], layer_units[i]) * 0.01
        biases[i] = np.zeros((1, layer_units[i]))

    return weights, biases

In [ ]:
weights, biases = initialize_weights(input_size, hidden_units, output_size)

In [ ]:
def relu_activation(Z):
    return np.maximum(0, Z)

In [ ]:
def softmax_activation(Z):
    expZ = np.exp(Z - np.max(Z))
    return expZ / expZ.sum(axis=1, keepdims=True)

In [ ]:
def relu_derivative(Z):
    return Z > 0

In [ ]:
def forward_pass(X, weights, biases):
    activations = []
    Z = X
    for i in range(len(weights)-1):
        Z = relu_activation(np.dot(Z, weights[i]) + biases[i])
        activations.append(Z)
    output = softmax_activation(np.dot(Z, weights[len(weights)-1]) + biases[len(weights)-1])
    activations.append(output)
    return activations

In [ ]:
def backward_pass(X, y, weights, biases, activations):
    gradients_weights = {}
    gradients_biases = {}
    L = len(weights)
    output = activations[-1]
    delta = output - y

    for i in reversed(range(L)):
        gradients_weights[i] = np.dot(activations[i-1].T, delta) if i > 0 else np.dot(X.T, delta)
        gradients_biases[i] = np.sum(delta, axis=0, keepdims=True)
        if i > 0:
            delta = np.dot(delta, weights[i].T) * relu_derivative(activations[i-1])

    return gradients_weights, gradients_biases

In [ ]:
for epoch in range(epochs):
    activations = forward_pass(X_train, weights, biases)
    gradients_weights, gradients_biases = backward_pass(X_train, y_train, weights, biases, activations)

    for i in range(len(weights)):
        weights[i] -= learning_rate * gradients_weights[i]
        biases[i] -= learning_rate * gradients_biases[i]

    train_output = activations[-1]
    loss = -np.mean(np.log(train_output[np.arange(len(y_train)), y_train.argmax(axis=1)]))
    predictions = np.argmax(train_output, axis=1)
    accuracy = np.mean(predictions == np.argmax(y_train, axis=1))

test_activations = forward_pass(X_test, weights, biases)
test_output = test_activations[-1]
test_predictions = np.argmax(test_output, axis=1)
print("Test Accuracy:", accuracy_score(np.argmax(y_test, axis=1), test_predictions))
print("Classification Report:")
print(classification_report(np.argmax(y_test, axis=1), test_predictions))

/tmp/ipykernel_33/2967250671.py:10: RuntimeWarning: divide by zero encountered in log
  loss = -np.mean(np.log(train_output[np.arange(len(y_train)), y_train.argmax(axis=1)]))
/tmp/ipykernel_33/2561999921.py:3: RuntimeWarning: invalid value encountered in divide
  return expZ / expZ.sum(axis=1, keepdims=True)


Test Accuracy: 0.1863425925925926
Classification Report:
              precision    recall  f1-score   support

           0       0.19      1.00      0.31       161
           1       0.00      0.00      0.00       224
           2       0.00      0.00      0.00       156
           3       0.00      0.00      0.00       144
           4       0.00      0.00      0.00       179

    accuracy                           0.19       864
   macro avg       0.04      0.20      0.06       864
weighted avg       0.03      0.19      0.06       864



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
